<a href="https://colab.research.google.com/github/dal7collab/google_collab_py/blob/main/indian_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT FILE FROM DRIVE

In [1]:
# Google User Authentication
from google.colab import auth
auth.authenticate_user()

In [2]:
# Google Credentials Authentication
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [3]:
# Import Google Sheet into Google Collab Data Object
file_name = 'indian_food' 
cn = gc.open(file_name).sheet1
sh = cn.get_all_values()
print(sh[0:1])

[['name', 'ingredients', 'diet', 'prep_time', 'cook_time', 'flavor_profile', 'course', 'state', 'region']]


# PANDAS

In [4]:
# Convert Google Sheet into Pandas Data Frame
import pandas as pd
import numpy  as np
df_sh = pd.DataFrame.from_records(sh)

In [5]:
# Convert First Row into Header
header = df_sh.iloc[0]        # Grab the first row for the header
df_sh  = df_sh[1:]            # Take the data less the header row
df_sh.columns = header        # Set the header row as the df header

In [6]:
# Obtain Data Frame General Info 
# Column names, Missing values, Data types
df_sh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 1 to 255
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            255 non-null    object
 1   ingredients     255 non-null    object
 2   diet            255 non-null    object
 3   prep_time       255 non-null    object
 4   cook_time       255 non-null    object
 5   flavor_profile  255 non-null    object
 6   course          255 non-null    object
 7   state           255 non-null    object
 8   region          255 non-null    object
dtypes: object(9)
memory usage: 18.1+ KB


In [7]:
# Copy dataframe
df = df_sh.copy(deep = True)

In [8]:
# Copy dataframe by specific column names
_df_ = df_sh.copy(deep = True)
cols = ['name', 'prep_time']
_df_ = _df_[cols]
_df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 1 to 255
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       255 non-null    object
 1   prep_time  255 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


In [9]:
# Retrieve First n (5) Rows
n = 2
df.head(n)
# df.tail(n)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
1,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
2,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West


In [10]:
# Describe Data Frame Series General Info
name = 'course'
df[name].describe()

count             255
unique              4
top       main course
freq              129
Name: course, dtype: object

In [11]:
# Obtain Data Frame Series Unique Values
name = 'course'
df[name].unique()

array(['dessert', 'main course', 'starter', 'snack'], dtype=object)

In [12]:
# Transform Column Type from Object to int32
df['prep_time'] = df['prep_time'].astype({'prep_time': 'int32'})

In [13]:
# Check again Data Frame Series General Info
name = 'prep_time'
df[name].describe()

count    255.000000
mean      31.105882
std       72.554409
min       -1.000000
25%       10.000000
50%       10.000000
75%       20.000000
max      500.000000
Name: prep_time, dtype: float64

In [14]:
# Function Casting Column Type
def cast_type(data_frame, column_name, type_name):
  data_frame[column_name] = data_frame[column_name].astype({column_name:type_name}) 
  print(column_name + ':ok')

In [15]:
# Function Run
column_name = 'cook_time'
cast_type(df, column_name, 'int32')
df[column_name].describe()

cook_time:ok


count    255.000000
mean      34.529412
std       48.265650
min       -1.000000
25%       20.000000
50%       30.000000
75%       40.000000
max      720.000000
Name: cook_time, dtype: float64

In [16]:
# Dataframe with float files
_df_ = pd.DataFrame()
_df_['File'] = np.random.random_sample((3))
# Turn float to object
rowIndex = 0
_df_.loc[rowIndex, 'File'] = 'Rand'
_df_.info()
# Turn object to float
header = _df_.iloc[0]        # Grab the first row for the header
_df_  = _df_[1:]            # Take the data less the header row
_df_.columns = header        # Set the header row as the df header
column_name = 'Rand'
cast_type(_df_, column_name, 'float')
print('======================================')
_df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    3 non-null      object
dtypes: object(1)
memory usage: 152.0+ bytes
Rand:ok
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 1 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rand    2 non-null      float64
dtypes: float64(1)
memory usage: 148.0 bytes


In [17]:
# Data Numerical Columns 'Simple' Stats
df.describe()

,prep_time,cook_time
count,255.000000,255.000000
mean,31.105882,34.529412
std,72.554409,48.265650
min,-1.000000,-1.000000
25%,10.000000,20.000000
50%,10.000000,30.000000
75%,20.000000,40.000000
max,500.000000,720.000000


In [18]:
# Data Categorical Columns 'Simple' Stats
df.describe(include=['O'])

,name,ingredients,diet,flavor_profile,course,state,region
count,255,255,255,255,255,255,255
unique,255,252,2,5,4,25,8
top,Papadum,"Gram flour, ghee, sugar",vegetarian,spicy,main course,Gujarat,West
freq,1,2,226,133,129,35,74


In [19]:
# Replace -1 values to np.nan values in specific column data frame
import numpy as np
df_rp = df.copy(deep = True)
df_rp['cook_time_none'] = df_rp['cook_time'].replace(to_replace = -1,
                                                          value = np.nan,
                                                        inplace = False)

In [20]:
# Source Dataframe Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 1 to 255
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            255 non-null    object
 1   ingredients     255 non-null    object
 2   diet            255 non-null    object
 3   prep_time       255 non-null    int32 
 4   cook_time       255 non-null    int32 
 5   flavor_profile  255 non-null    object
 6   course          255 non-null    object
 7   state           255 non-null    object
 8   region          255 non-null    object
dtypes: int32(2), object(7)
memory usage: 16.1+ KB


In [21]:
# Current Dataframe Info after -1 values replace to NaN
df_rp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 1 to 255
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            255 non-null    object 
 1   ingredients     255 non-null    object 
 2   diet            255 non-null    object 
 3   prep_time       255 non-null    int32  
 4   cook_time       255 non-null    int32  
 5   flavor_profile  255 non-null    object 
 6   course          255 non-null    object 
 7   state           255 non-null    object 
 8   region          255 non-null    object 
 9   cook_time_none  227 non-null    float64
dtypes: float64(1), int32(2), object(7)
memory usage: 18.1+ KB


In [22]:
# Replace -1 Values to np.nan values in Dataframe
import numpy as np
df_rp = df.copy(deep = True)
df_rp.replace(to_replace = [-1, '-1'],
                   value = np.nan,
                 inplace = True)

In [23]:
# Count NaN rows in Dataframe
name = 'state'
s = df_rp.loc[df_rp[name].isna()]
len(s.index)

24

# MISSING VALUES REPLACEMENT

In [24]:
# Function replaces np.nan values in all data frame with relevant statistics (mean, mode)
def rep_missings(data_frame):
    d = data_frame.copy(deep = True)
    for c in data_frame:
        if (data_frame[c].dtype == 'int32'):
            if data_frame[c].isnull().values.any():
                m = data_frame[c].dropna().mean()
                d[c].fillna(m, inplace = True)
        else:   
            if data_frame[c].isnull().values.any():
              m = data_frame[c].dropna().mode()[0]
              d[c].fillna(m, inplace = True)              
    return d

In [25]:
# Replace np.nan values in all data frame with relevant statistics (mean, mode)
df_rp = rep_missings(df_rp)

In [26]:
name = 'region'
a = df_rp[name].unique()
a
# a[0]
# a[a.size - 1]

array(['East', 'West', 'North', 'North East', 'South', 'Central', ''],
      dtype=object)

# EXPORT FILE TO DRIVE

In [27]:
# Mount Google drive in Google Collab
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [28]:
# Export data_frame into Google drive in comma separated format
df_rp.to_csv('/drive/My Drive/indian_food_nulls.csv')
print("File was exported succesfully.")

File was exported succesfully.


# BI DASHBOARDS OPTIMIZATION

In [29]:
name = 'region'
a = df_rp[name].unique()
a[2]

'North'

In [30]:
# BI INSIGHTS ROBOT
def bi_robot(dataframe, dimensions, measure):
  # Student WORK 
  # Lecturer first preparations, hints and thoughts
  # This function needs to return anomalous tuples which can be serve
  # as default values in BI dashboards and further BI dashboards conclusions
  t = {}
  for d in dimensions:
    du = dataframe[d].unique() 
    t[d] = du[0] 

  for v in t:
    key       = v
    value     = [t[v]]
    dataframe = dataframe.loc[dataframe[key].isin(value)] 
    stat      = dataframe[measure].mean() 
  
  print('---------------------')
  print(t, end = '')  
  print(' Mean:' + "{:.2f}".format(stat))
  print('---------------------')
  return t


In [31]:
df         = df_rp.copy(deep = True)
filters    = ['diet', 'course']
measure    = 'prep_time'
df = bi_robot(df, filters, measure)
df

---------------------
{'diet': 'vegetarian', 'course': 'dessert'} Mean:28.08
---------------------


{'course': 'dessert', 'diet': 'vegetarian'}